In [1]:
!pip install pandas matplotlib numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 295.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 240.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 401.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 273.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 297.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 355.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━ 10/11 [matplotlib]^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━ 10/11 [matplotlib]


In [3]:
# 기본 재시도
!pip install pandas matplotlib numpy

# 네트워크가 불안정할 때 (다운로드 재시도 횟수 & 타임아웃 늘리기)
!pip install pandas matplotlib numpy --retries 5 --timeout 120

# 캐시 문제 회피
!pip install pandas matplotlib numpy --no-cache-dir

# 권한 이슈가 있을 때(Windows 관리자 권한 터미널 또는 가상환경 내에서)
!python -m pip install --upgrade pip
!python -m pip install pandas matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.1 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

[sudo] password for user5: 
sudo: a password is required
^C
[sudo] password for user5: 

In [8]:
# filter_joy_json.py
import json
import os
import sys

# === 설정값 ===
INPUT_JSON   = r"/workspace/new_data/json/unicode_decoded_happy_data.json"      # 원본 JSON (배열 JSON 또는 NDJSON 모두 지원)
IMAGE_SOURCE = r"/workspace/new_data/img"  # '기쁨' 폴더 경로 또는 파일명 목록 txt 경로
OUTPUT_JSON  = "metadata_filtered_joy.json"

# 폴더 탐색 옵션
RECURSIVE_SCAN = False   # True로 바꾸면 하위 폴더까지 모두 포함
ALLOW_EXTENSIONS = None  # 예: {"jpg", "jpeg", "png"} 로 제한하려면 지정 (None이면 모든 확장자 포함)

def load_json_safely(path):
    """JSON이 배열인지, NDJSON(줄 단위)인지 자동 감지하여 리스트로 반환"""
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()
    if not text:
        return []
    try:
        data = json.loads(text)
        if isinstance(data, list):
            return data
        return [data]  # 단일 오브젝트면 리스트로 감싸기
    except json.JSONDecodeError:
        # NDJSON 파싱
        records = []
        with open(path, "r", encoding="utf-8") as f2:
            for line in f2:
                line = line.strip()
                if not line:
                    continue
                try:
                    obj = json.loads(line)
                    records.append(obj)
                except json.JSONDecodeError:
                    # 깨진 라인은 건너뜀
                    pass
        return records

def normalize_filename(name):
    """경로 제거 + 트림 + 소문자 변환"""
    if not isinstance(name, str):
        return ""
    base = os.path.basename(name.strip())
    return base.lower()

def list_filenames_from_dir(dir_path, recursive=False, allow_exts=None):
    """폴더에서 파일명 목록 생성 (basename만, 소문자)"""
    result = set()
    if recursive:
        for root, dirs, files in os.walk(dir_path):
            for fn in files:
                if allow_exts:
                    ext = os.path.splitext(fn)[1].lower().lstrip(".")
                    if ext not in allow_exts:
                        continue
                result.add(normalize_filename(fn))
    else:
        # 상위 폴더만
        for fn in os.listdir(dir_path):
            full = os.path.join(dir_path, fn)
            if os.path.isfile(full):
                if allow_exts:
                    ext = os.path.splitext(fn)[1].lower().lstrip(".")
                    if ext not in allow_exts:
                        continue
                result.add(normalize_filename(fn))
    return result

def load_image_names(source_path):
    """
    source_path가 디렉터리면 폴더 스캔,
    파일이면 줄단위로 읽어서 집합 반환
    """
    if not os.path.exists(source_path):
        print(f"[ERROR] 존재하지 않는 경로: {source_path}")
        sys.exit(1)

    # 디렉터리 처리
    if os.path.isdir(source_path):
        print(f"[INFO] 폴더에서 파일명 읽는 중: {source_path} (recursive={RECURSIVE_SCAN})")
        names = list_filenames_from_dir(
            source_path,
            recursive=RECURSIVE_SCAN,
            allow_exts=ALLOW_EXTENSIONS
        )
        if not names:
            print("[WARN] 폴더에서 발견된 파일이 없습니다.")
        return names

    # 텍스트 파일 처리
    joy_names = set()
    with open(source_path, "r", encoding="utf-8") as f:
        for line in f:
            name = line.strip()
            if not name:
                continue
            joy_names.add(normalize_filename(name))
    if not joy_names:
        print("[WARN] 텍스트 파일에서 읽은 파일명이 없습니다.")
    return joy_names

def main():
    # 1) 이미지 파일명 집합 로드 (폴더 or 텍스트 파일)
    joy_names = load_image_names(IMAGE_SOURCE)

    # 2) 원본 JSON 로드
    if not os.path.exists(INPUT_JSON):
        print(f"[ERROR] JSON 파일이 없습니다: {INPUT_JSON}")
        sys.exit(1)
    records = load_json_safely(INPUT_JSON)
    total = len(records)

    # 3) 필터링: faceExp_uploader == "기쁨" AND filename ∈ joy_names
    kept = []
    dropped_missing_filename = 0
    dropped_not_joy_label = 0
    dropped_not_in_folder = 0

    for rec in records:
        fn = normalize_filename(rec.get("filename", ""))
        label = str(rec.get("faceExp_uploader", "")).strip()
        if not fn:
            dropped_missing_filename += 1
            continue
        if label != "기쁨":
            dropped_not_joy_label += 1
            continue
        if fn not in joy_names:
            dropped_not_in_folder += 1
            continue
        kept.append(rec)

    # 4) 저장
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(kept, f, ensure_ascii=False, indent=2)

    # 5) 통계 출력
    print("===== 결과 요약 =====")
    print(f"총 레코드: {total}")
    print(f"유지(기쁨+파일매칭): {len(kept)}")
    print(f"제외 - filename 없음: {dropped_missing_filename}")
    print(f"제외 - 기쁨 아님: {dropped_not_joy_label}")
    print(f"제외 - 기쁨 폴더에 파일 없음: {dropped_not_in_folder}")
    print(f"출력 파일: {OUTPUT_JSON}")

if __name__ == "__main__":
    main()


[INFO] 폴더에서 파일명 읽는 중: /workspace/new_data/img (recursive=False)


===== 결과 요약 =====
총 레코드: 7499
유지(기쁨+파일매칭): 7499
제외 - filename 없음: 0
제외 - 기쁨 아님: 0
제외 - 기쁨 폴더에 파일 없음: 0
출력 파일: metadata_filtered_joy.json


In [ ]:
!python -m pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [8]:
!python -m pip install xlrd

In [10]:
# 가상환경이 활성화된 터미널에서
!python -m pip install --upgrade pip
!python -m pip install openpyxl

In [ ]:


import os
import json
import zipfile
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ========== 1) 경로 설정 ==========
# ⚠️ Windows 경로는 r"..." 또는 "/" 사용
JSON_PATH = r"/workspace/new_data/json/unicode_decoded_happy_data.json" 

# (선택) 이미지 폴더 또는 ZIP: 지정하면 '매칭된 이미지' 기준 그래프도 추가 저장
IMG_DIR_OR_ZIP = r"/workspace/new_data/img"
USE_UPLOADER_AS_TIE_BREAK = True   # 동률 시 업로더 라벨을 먼저 고려할지

# ========== 2) 폰트/유틸 ==========
def ensure_font():
    """한글 폰트 설정(Windows: 맑은 고딕, macOS: AppleGothic, 기타: DejaVu Sans)"""
    import platform
    system = platform.system()
    if system == "Windows":
        plt.rcParams['font.family'] = 'Malgun Gothic'
    elif system == "Darwin":
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'DejaVu Sans'
    plt.rcParams['axes.unicode_minus'] = False

def to_base_name(path_or_name: str) -> str:
    """경로/확장자를 제거하고, 소문자로 통일한 베이스 이름 반환."""
    if path_or_name is None:
        return None
    name = os.path.basename(str(path_or_name))
    if "." in name:
        name = ".".join(name.split(".")[:-1])  # 마지막 점 기준 확장자 제거
    return name.strip().lower()

def list_image_basenames_from_dir_or_zip(dir_or_zip: str) -> set:
    """폴더 또는 zip에서 .jpg/.jpeg 파일 베이스 이름 set 수집."""
    if not dir_or_zip:
        return set()
    exts = {".jpg", ".jpeg"}
    basenames = set()
    p = str(dir_or_zip)
    if p.lower().endswith(".zip"):
        with zipfile.ZipFile(p, "r") as zf:
            for nm in zf.namelist():
                bn = os.path.basename(nm)
                ext = os.path.splitext(bn)[1].lower()
                if ext in exts:
                    basenames.add(to_base_name(bn))
    else:
        for root, _, files in os.walk(p):
            for f in files:
                ext = os.path.splitext(f)[1].lower()
                if ext in exts:
                    basenames.add(to_base_name(f))
    return basenames

# 영어/한글 라벨 표준화 매핑(원하면 자유롭게 수정하세요)
LABEL_MAP = {
    "neutral": "중립", "happy": "기쁨", "joy": "기쁨",
    "sad": "슬픔", "sadness": "슬픔",
    "angry": "분노", "anger": "분노",
    "surprise": "경악", "surprised": "경악",
    "fear": "공포", "disgust": "혐오", "contempt": "경멸",
    "confused": "혼란", "bored": "지루함",

    # 한국어 원라벨(그대로)
    "중립": "중립", "기쁨": "기쁨", "슬픔": "슬픔", "분노": "분노",
    "경악": "경악", "공포": "공포", "혐오": "혐오", "경멸": "경멸", "혼란": "혼란", "지루함": "지루함",
}

# 동률 우선순위(앞쪽일수록 우선)
PRIORITY_ORDER = ["기쁨", "중립", "슬픔", "분노", "경악", "공포", "혐오", "경멸", "혼란", "지루함"]

def normalize_label(lbl: str) -> str:
    """라벨 표준화: 소문자/trim → 한국어로 매핑. 매핑 없으면 원문 유지."""
    if lbl is None:
        return None
    s = str(lbl).strip()
    key = s.lower()
    return LABEL_MAP.get(key, s)

def parse_faceexp_any(v):
    """
    faceExp 값 파싱: 문자열 | dict | list 모두 지원
    - 문자열: 직접 매핑
    - dict: {'faceExp': '기쁨'} 또는 {'happy': 0.9, 'sad': 0.1} → faceExp 값 또는 최고 점수 라벨
    - list: [{'label': '기쁨', 'score': 0.9}, ...] → 최고 점수 라벨
    반환: 표준화된 문자열 라벨 또는 None
    """
    if v is None:
        return None

    if isinstance(v, str):
        return normalize_label(v)

    if isinstance(v, dict):
        # 명시 키가 있을 때
        if 'faceExp' in v:
            return normalize_label(v.get('faceExp'))
        # 점수 dict일 때
        scores = {}
        for k, val in v.items():
            try:
                scores[normalize_label(k)] = float(val)
            except (TypeError, ValueError):
                pass
        if scores:
            return max(scores.items(), key=lambda kv: kv[1])[0]
        return None

    if isinstance(v, list):
        scores = {}
        for item in v:
            if isinstance(item, dict):
                label = item.get('label') or item.get('faceExp') or item.get('emotion') or item.get('emo')
                score = item.get('score') or item.get('prob') or item.get('confidence')
                if label is not None and score is not None:
                    try:
                        scores[normalize_label(label)] = float(score)
                    except (TypeError, ValueError):
                        pass
        if scores:
            return max(scores.items(), key=lambda kv: kv[1])[0]
        return None

    return None

def majority_vote(labels, uploader_top=None, use_uploader=USE_UPLOADER_AS_TIE_BREAK):
    """
    labels: [A, B, C] 표준화된 문자열(또는 None)
    uploader_top: 업로더 최상위 라벨(선택)
    규칙:
      - 2표 이상이면 그 라벨
      - 1-1-1 동률이면:
          (옵션) uploader_top이 있고 사용 설정이면 → uploader_top
          중립 포함 시 → 중립
          아니면 PRIORITY_ORDER에서 가장 앞선 라벨
    """
    labels = [l for l in labels if l is not None]
    if not labels:
        return None
    cnt = Counter(labels)
    top_lbl, top_n = cnt.most_common(1)[0]
    if top_n >= 2:
        return top_lbl

    uniq = set(labels)

    if use_uploader and uploader_top and uploader_top in uniq:
        return uploader_top

    if "중립" in uniq:
        return "중립"

    for p in PRIORITY_ORDER:
        if p in uniq:
            return p

    return labels[0]  # 예외: 매핑 밖 레이블이면 임의 반환

# ========== 3) 데이터 로드 ==========
ensure_font()

with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

# 레코드 리스트 추출
records = []
if isinstance(raw, list):
    records = raw
elif isinstance(raw, dict):
    if 'data' in raw and isinstance(raw['data'], list):
        records = raw['data']
    else:
        for v in raw.values():
            if isinstance(v, list):
                records = v
                break
if not records:
    raise ValueError("JSON에서 분석할 레코드를 찾지 못했습니다. 리스트 형태여야 합니다.")

df = pd.DataFrame.from_records(records)

# 필수 컬럼 체크
for col in ['filename', 'annot_A', 'annot_B', 'annot_C']:
    if col not in df.columns:
        raise ValueError(f"JSON에 '{col}' 컬럼이 없습니다.")

has_uploader = 'faceExp_uploader' in df.columns

# ========== 4) 파일명/이미지 매칭 ==========
df['base_name'] = df['filename'].apply(to_base_name)
img_basenames = list_image_basenames_from_dir_or_zip(IMG_DIR_OR_ZIP) if IMG_DIR_OR_ZIP else set()
df['has_image'] = df['base_name'].isin(img_basenames) if img_basenames else True  # 이미지 경로 없으면 모두 True

# ========== 5) 라벨 파싱 & 다수결 ==========
def get_annot_label(row, key):
    return normalize_label(parse_faceexp_any(row.get(key)))

def get_uploader_top(row):
    v = row.get('faceExp_uploader')
    return normalize_label(parse_faceexp_any(v))

df['A'] = df.apply(lambda r: get_annot_label(r, 'annot_A'), axis=1)
df['B'] = df.apply(lambda r: get_annot_label(r, 'annot_B'), axis=1)
df['C'] = df.apply(lambda r: get_annot_label(r, 'annot_C'), axis=1)
df['uploader_top'] = df.apply(lambda r: get_uploader_top(r) if has_uploader else None, axis=1)

df['final_label'] = df.apply(
    lambda r: majority_vote([r['A'], r['B'], r['C']], uploader_top=r['uploader_top']),
    axis=1
)

# ========== 6) 분포 집계 ==========
def counts_annot_flat(sub_df):
    """다수결 전: annot A/B/C 전체를 낱개 표본으로 평탄화해 집계"""
    flat = []
    vals = sub_df[['A','B','C']].values
    for a, b, c in vals:
        for v in (a, b, c):
            if v is not None:
                flat.append(v)
    return pd.Series(flat).value_counts().sort_index()

def counts_final(sub_df):
    return sub_df['final_label'].dropna().value_counts().sort_index()

# 전체
counts_pre_all = counts_annot_flat(df)            # 수정 전
counts_post_all = counts_final(df)                # 수정 후

# 매칭만(이미지 존재)
matched_df = df[df['has_image']]
counts_pre_matched = counts_annot_flat(matched_df)
counts_post_matched = counts_final(matched_df)

# ========== 7) 시각화(그룹 막대: 수정 전 vs 수정 후) ==========
os.makedirs("figures", exist_ok=True)

def plot_group_compare(pre_counts, post_counts, title, outfile):
    labels_union = sorted(set(pre_counts.index).union(set(post_counts.index)))
    comp = pd.DataFrame({
        '감정 라벨': labels_union,
        '수정 전': [int(pre_counts.get(lbl, 0)) for lbl in labels_union],
        '수정 후': [int(post_counts.get(lbl, 0)) for lbl in labels_union],
    })
    # 보기 좋게 총합 내림차순
    comp['총합'] = comp['수정 전'] + comp['수정 후']
    comp = comp.sort_values('총합', ascending=False).drop(columns=['총합'])

    x = np.arange(len(comp['감정 라벨']))
    w = 0.38

    plt.figure(figsize=(max(8, len(x)*0.9), 6))
    plt.bar(x - w/2, comp['수정 전'], width=w, color='#b0bec5', label='수정 전')
    plt.bar(x + w/2, comp['수정 후'], width=w, color='#26a69a', label='수정 후')

    plt.xticks(x, comp['감정 라벨'], rotation=25, ha='right')
    plt.ylabel('Count')
    plt.title(title)
    plt.legend()

    # 값 라벨(옵션)
    for xi, v in zip(x - w/2, comp['수정 전']):
        if v > 0:
            plt.text(xi, v + max(1, v*0.01), str(v), ha='center', va='bottom', fontsize=9, color='#455a64')
    for xi, v in zip(x + w/2, comp['수정 후']):
        if v > 0:
            plt.text(xi, v + max(1, v*0.01), str(v), ha='center', va='bottom', fontsize=9, color='#00695c')

    plt.tight_layout()
    plt.savefig(outfile, dpi=150)
    plt.close()

# 저장(3번째 예시 스타일)
plot_group_compare(counts_pre_all, counts_post_all,
                   "faceExp 분포 비교 (수정 전 vs 수정 후, 전체)",
                   os.path.join("figures", "faceExp_distribution_compare_all.png"))

plot_group_compare(counts_pre_matched, counts_post_matched,
                   "faceExp 분포 비교 (수정 전 vs 수정 후, 매칭)",
                   os.path.join("figures", "faceExp_distribution_compare_matched.png"))

# ========== 8) 텍스트 요약 출력 ==========
def print_summary_block(title, pre_counts, post_counts):
    print(f"[{title}]")
    print("  ┌ 수정 전(annot A/B/C 전체) ───────────────")
    if pre_counts.empty: print("    (데이터 없음)")
    else:
        for lbl, val in pre_counts.items():
            print(f"    {lbl:>4s} : {val}")
    print("  └──────────────────────────────────────────")
    print("  ┌ 수정 후(다수결 최종 레이블) ─────────────")
    if post_counts.empty: print("    (데이터 없음)")
    else:
        for lbl, val in post_counts.items():
            print(f"    {lbl:>4s} : {val}")
    print("  └──────────────────────────────────────────\n")

print_summary_block("전체", counts_pre_all, counts_post_all)
print_summary_block("매칭", counts_pre_matched, counts_post_matched)
print(f"총 레코드 수: {len(df)} / 매칭된 레코드 수: {int(df['has_image'].sum())}")

# ========== 9) 결과 CSV 저장 ==========
out_csv = os.path.join("figures", "final_labels.csv")
df_out = df[['filename','A','B','C','uploader_top','final_label','has_image']]
df_out.to_csv(out_csv, index=False, encoding='utf-8-sig')
print(f"완료: 그래프 2종과 CSV가 'figures' 폴더에 저장되었습니다.")


/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 49256 (\N{HANGUL SYLLABLE BBEUM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 51473 (\N{HANGUL SYLLABLE JUNG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 47549 (\N{HANGUL SYLLABLE RIB}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 45817 (\N{HANGUL SYLLABLE DANG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 54889 (\N{HANGUL SYLLABLE HWANG}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_75100/379102348.py:277: UserWarning: Glyph 48520 (\N{HANGUL SYLLABLE BUL}) missing from font(s) DejaVu Sans.
 

[전체]
  ┌ 수정 전(annot A/B/C 전체) ───────────────
      기쁨 : 21614
      당황 : 190
      분노 : 53
      불안 : 99
      상처 : 82
      슬픔 : 83
    알수없음 : 2
      중립 : 374
  └──────────────────────────────────────────
  ┌ 수정 후(다수결 최종 레이블) ─────────────
      기쁨 : 7371
      당황 : 26
      분노 : 6
      불안 : 8
      상처 : 2
      슬픔 : 9
    알수없음 : 1
      중립 : 76
  └──────────────────────────────────────────

[매칭]
  ┌ 수정 전(annot A/B/C 전체) ───────────────
      기쁨 : 21614
      당황 : 190
      분노 : 53
      불안 : 99
      상처 : 82
      슬픔 : 83
    알수없음 : 2
      중립 : 374
  └──────────────────────────────────────────
  ┌ 수정 후(다수결 최종 레이블) ─────────────
      기쁨 : 7371
      당황 : 26
      분노 : 6
      불안 : 8
      상처 : 2
      슬픔 : 9
    알수없음 : 1
      중립 : 76
  └──────────────────────────────────────────

총 레코드 수: 7499 / 매칭된 레코드 수: 7499
완료: 그래프 2종과 CSV가 'figures' 폴더에 저장되었습니다.


In [17]:
# -*- coding: utf-8 -*-

import os
import json
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd

# ========== 1) 경로 ==========
JSON_PATH = r"/workspace/new_data/json/unicode_decoded_happy_data.json"
EXCEL_PATH = r"/workspace/new_data/label/json_img_matching_happy.xlsx"  # .xlsx/.xls/.csv 모두 지원

EXCEL_SHEET_NAME = 0
EXCEL_FILENAME_COL = "gudt"
EXCEL_JUDGE_COL = "judge"

# 출력 파일명은 원본 JSON 이름을 기준으로 생성
SPLIT_CHANGED_SUFFIX   = "_changed"    # 바뀐 78건
SPLIT_UNCHANGED_SUFFIX = "_unchanged"  # 나머지 7421건
UPDATED_JSON_SUFFIX    = "_updated"    # (전체 업데이트본이 필요할 때)

# ========== 2) 헬퍼 ==========
def to_base_name(path_or_name: str) -> str:
    """경로/확장자 제거 + 소문자 통일한 베이스 이름."""
    if path_or_name is None:
        return None
    name = os.path.basename(str(path_or_name)).strip()
    if "." in name:
        name = ".".join(name.split(".")[:-1])
    return name.lower()

def normalize_judge(lbl: str) -> str:
    """judge 라벨을 '기쁨' 또는 '중립'으로 정규화."""
    if lbl is None or (isinstance(lbl, float) and pd.isna(lbl)):
        return None
    s = str(lbl).strip().lower()
    mapping = {
        "기쁨": "기쁨", "행복": "기쁨", "joy": "기쁨", "happy": "기쁨",
        "중립": "중립", "neutral": "중립"
    }
    return mapping.get(s, s)

def faceexp_top_label(value):
    """
    faceExp_uploader의 최상위 라벨만 추출(문자열/딕셔너리/리스트 모두 대응).
    - 문자열: 그대로
    - dict: 최고 점수 키
    - list[dict]: 최고 점수 label
    """
    if value is None:
        return None
    if isinstance(value, str):
        return value.strip()
    if isinstance(value, dict):
        try:
            return max(value.items(), key=lambda kv: float(kv[1]))[0]
        except Exception:
            return None
    if isinstance(value, list):
        scores = {}
        for item in value:
            if isinstance(item, dict):
                label = item.get('label') or item.get('faceExp') or item.get('emotion') or item.get('emo')
                score = item.get('score') or item.get('prob') or item.get('confidence')
                if label is not None and score is not None:
                    try:
                        scores[str(label).strip()] = float(score)
                    except Exception:
                        pass
        if scores:
            return max(scores.items(), key=lambda kv: kv[1])[0]
    return None

def load_table(path: str, sheet_name=0) -> pd.DataFrame:
    """
    .xlsx/.xls/.csv 자동 판별하여 로드.
    - .xlsx/.xlsm: openpyxl 필요 (pip install openpyxl)
    - .xls: xlrd 필요 (pip install xlrd)
    - .csv: 엔진 불필요
    """
    ext = Path(path).suffix.lower()
    if ext in [".xlsx", ".xlsm"]:
        try:
            return pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")
        except ImportError:
            raise ImportError("openpyxl가 필요합니다. 설치 후 실행하거나, 엑셀을 CSV로 저장해 주세요.")
    elif ext == ".xls":
        try:
            return pd.read_excel(path, sheet_name=sheet_name, engine="xlrd")
        except ImportError:
            raise ImportError("xlrd가 필요합니다. 설치 후 실행하거나, 엑셀을 CSV로 저장해 주세요.")
    elif ext in [".csv", ".tsv"]:
        sep = "," if ext == ".csv" else "\t"
        return pd.read_csv(path, sep=sep, encoding="utf-8")
    else:
        raise ValueError(f"지원하지 않는 파일 형식: {ext}")

# ========== 3) 데이터 로드 ==========
# JSON 로드
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)

# 레코드 리스트 추출 (list 또는 dict 래핑 모두 대응)
records = []
if isinstance(raw, list):
    records = raw
elif isinstance(raw, dict):
    if 'data' in raw and isinstance(raw['data'], list):
        records = raw['data']
    else:
        for v in raw.values():
            if isinstance(v, list):
                records = v
                break
if not records:
    raise ValueError("JSON에서 분석할 레코드를 찾지 못했습니다. 리스트 형태여야 합니다.")

df = pd.DataFrame.from_records(records)
if 'filename' not in df.columns:
    raise ValueError("JSON에 'filename' 컬럼이 없습니다.")

# 기존 top label (비교 기준)
before_top = df['faceExp_uploader'].apply(faceexp_top_label) if 'faceExp_uploader' in df.columns else pd.Series([None]*len(df))

# JSON 파일명 베이스 이름
df['base_name_json'] = df['filename'].apply(to_base_name)

# 엑셀/CSV 로드
xlsx = load_table(EXCEL_PATH, sheet_name=EXCEL_SHEET_NAME)
for col in [EXCEL_FILENAME_COL, EXCEL_JUDGE_COL]:
    if col not in xlsx.columns:
        raise ValueError(f"엑셀/CSV에 '{col}' 컬럼이 없습니다. 실제 컬럼: {list(xlsx.columns)}")

# 엑셀 파일명/라벨 정규화
xlsx['base_name_excel'] = xlsx[EXCEL_FILENAME_COL].astype(str).apply(to_base_name)
xlsx['judge_norm'] = xlsx[EXCEL_JUDGE_COL].apply(normalize_judge)
xlsx_valid = xlsx.dropna(subset=['base_name_excel', 'judge_norm']).copy()
xlsx_valid = xlsx_valid[xlsx_valid['judge_norm'].isin(['기쁨', '중립'])]

# 같은 파일명이 엑셀에 여러 번 있을 때 → 다수결로 결정(동수면 중립 우선)
vote_map = {}
for basename, grp in xlsx_valid.groupby('base_name_excel'):
    counts = grp['judge_norm'].value_counts()
    if len(counts) == 0:
        continue
    if len(counts) == 1:
        final = counts.index[0]
    else:
        most = counts.max()
        winners = counts[counts == most].index.tolist()
        final = '중립' if '중립' in winners else winners[0]
    vote_map[basename] = final

# ========== 4) 업데이트 & 변경 여부 산출 ==========
df['mapped_judge'] = df['base_name_json'].map(vote_map)

# 이 줄을 True로 바꾸면 "매핑만 있으면 변경으로 간주(=180건)"가 됩니다.
USE_STRICT_CHANGE_CRITERIA = True

if USE_STRICT_CHANGE_CRITERIA:
    # 이전 코드와 동일: 최상위 라벨이 다를 때만 '실제 교체'
    changed_mask = df['mapped_judge'].notna() & (before_top.fillna("__NA__") != df['mapped_judge'])
else:
    # 매핑된 모든 건을 '변경'으로 간주
    changed_mask = df['mapped_judge'].notna()

# 업데이트된 값(매핑이 있으면 교체)
def replace_uploader(old_val, mapped):
    return mapped if mapped is not None else old_val

df['faceExp_uploader_updated'] = [
    replace_uploader(o, m) for o, m in zip(df.get('faceExp_uploader', pd.Series([None]*len(df))), df['mapped_judge'])
]

# ========== 5) 분리 저장 ==========
# 원본 구조를 유지할 필요가 있으면, 여기서 dict 래핑을 복원할 수 있으나
# 요구사항에 따라 '리스트 JSON'으로 저장합니다.

# 업데이트된 전체 레코드(리스트)
updated_records = []
for i, rec in enumerate(records):
    rec2 = dict(rec)
    rec2['faceExp_uploader'] = df.loc[i, 'faceExp_uploader_updated']
    updated_records.append(rec2)

# 분리: 변경/비변경
changed_indices = df.index[changed_mask].tolist()
unchanged_indices = df.index[~changed_mask].tolist()

changed_records   = [updated_records[i] for i in changed_indices]
unchanged_records = [updated_records[i] for i in unchanged_indices]

# 파일 경로
json_path = Path(JSON_PATH)
out_changed   = json_path.with_name(json_path.stem + SPLIT_CHANGED_SUFFIX   + json_path.suffix)
out_unchanged = json_path.with_name(json_path.stem + SPLIT_UNCHANGED_SUFFIX + json_path.suffix)
out_updated   = json_path.with_name(json_path.stem + UPDATED_JSON_SUFFIX    + json_path.suffix)  # 참고: 전체 업데이트본

# 저장
with open(out_changed, "w", encoding="utf-8") as f:
    json.dump(changed_records, f, ensure_ascii=False, indent=2)

with open(out_unchanged, "w", encoding="utf-8") as f:
    json.dump(unchanged_records, f, ensure_ascii=False, indent=2)

# (선택) 전체 업데이트본도 저장하고 싶으면 아래 주석 해제
with open(out_updated, "w", encoding="utf-8") as f:
    json.dump(updated_records, f, ensure_ascii=False, indent=2)

# (부가) 변경된 파일명 목록 저장
changed_filenames = [records[i].get('filename') for i in changed_indices]
with open(json_path.with_name(json_path.stem + "_changed_filenames.json"), "w", encoding="utf-8") as f:
    json.dump(changed_filenames, f, ensure_ascii=False, indent=2)

# 검증 출력
print(f"총 레코드 수: {len(df)}")
print(f"매핑 존재 수: {int(df['mapped_judge'].notna().sum())}")
print(f"실제 교체(변경) 수: {len(changed_records)} → 저장: {out_changed.name}")
print(f"비변경 수: {len(unchanged_records)} → 저장: {out_unchanged.name}")
print(f"전체 업데이트본 저장: {out_updated.name}")


총 레코드 수: 7499
매핑 존재 수: 180
실제 교체(변경) 수: 78 → 저장: unicode_decoded_happy_data_changed.json
비변경 수: 7421 → 저장: unicode_decoded_happy_data_unchanged.json
전체 업데이트본 저장: unicode_decoded_happy_data_updated.json


3710필터링

In [20]:
import os
import json
import csv
import statistics

# ===== 설정 =====
INPUT_JSON = "/workspace/new_data/split_half/happy_half.json"  # ← 수천 개 이미지가 들어있는 단일 JSON 파일
OUTPUT_CSV  = "./bbox_agg_median_per_image.csv"

def normalize_item_to_annots(item, fallback_image=None):
    """
    한 '이미지 단위' 객체에서 annot_*의 boxes를 추출하여 통일된 형태로 반환.
    반환: (image_name, list of boxes dicts [{minX,...}, ...])
    """
    image_name = item.get("image", fallback_image)
    boxes_list = []

    def maybe_push(annot_val):
        if not isinstance(annot_val, dict):
            return
        boxes = annot_val.get("boxes")
        if not isinstance(boxes, dict):
            return
        try:
            minX = float(boxes["minX"]); minY = float(boxes["minY"])
            maxX = float(boxes["maxX"]); maxY = float(boxes["maxY"])
        except (KeyError, ValueError, TypeError):
            return
        boxes_list.append({"minX": minX, "minY": minY, "maxX": maxX, "maxY": maxY})

    # 케이스 1) {"annotations": {annot_A:{boxes:...}, annot_B:{...}, ...}}
    if "annotations" in item and isinstance(item["annotations"], dict):
        for _, annot_val in item["annotations"].items():
            maybe_push(annot_val)
    else:
        # 케이스 2) 루트에 바로 annot_A/B/C가 있는 경우
        # 또는 item 자체가 단일 annot 구조(바로 boxes 포함)인 경우까지 커버
        found_any = False
        for key, val in item.items():
            if isinstance(val, dict) and "boxes" in val:
                maybe_push(val)
                found_any = True
        if not found_any and "boxes" in item:
            # item 자체가 하나의 annot일 때
            maybe_push(item)

    return image_name, boxes_list

def process_large_json(input_json):
    """
    대용량 단일 JSON(리스트 또는 딕셔너리) 전체를 읽어
    이미지별로 A/B/C의 min/max 중앙값을 계산하고 center/width/height까지 계산해 행 목록 반환.
    """
    with open(input_json, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []

    # 최상위가 리스트인 경우: 각 원소가 이미지 단위라고 가정
    if isinstance(data, list):
        for idx, item in enumerate(data):
            if not isinstance(item, dict):
                continue
            image_name, boxes_list = normalize_item_to_annots(item, fallback_image=f"img_{idx}")
            # A/B/C 등에서 모인 박스가 없으면 스킵
            if not boxes_list:
                continue

            # 좌표별 리스트 만들기
            minXs = [b["minX"] for b in boxes_list]
            minYs = [b["minY"] for b in boxes_list]
            maxXs = [b["maxX"] for b in boxes_list]
            maxYs = [b["maxY"] for b in boxes_list]

            agg_minX = statistics.median(minXs)
            agg_minY = statistics.median(minYs)
            agg_maxX = statistics.median(maxXs)
            agg_maxY = statistics.median(maxYs)

            centerX = (agg_minX + agg_maxX) / 2.0
            centerY = (agg_minY + agg_maxY) / 2.0
            width   = agg_maxX - agg_minX
            height  = agg_maxY - agg_minY

            rows.append({
                "image": image_name,
                "agg_minX": agg_minX, "agg_minY": agg_minY,
                "agg_maxX": agg_maxX, "agg_maxY": agg_maxY,
                "centerX": centerX, "centerY": centerY,
                "width": width, "height": height,
            })

    # 최상위가 딕셔너리인 경우:
    # - 형태 A) {"images": [...]} 같은 컬렉션 키 아래 리스트
    # - 형태 B) 루트에 이미지 단위 딕셔너리들이 key별로 들어 있는 경우(덜 일반적)
    elif isinstance(data, dict):
        # 형태 A 우선 시도
        candidate_keys = ["images", "data", "items", "records", "annotations_list"]
        img_list = None
        for k in candidate_keys:
            if k in data and isinstance(data[k], list):
                img_list = data[k]
                break

        if img_list is not None:
            for idx, item in enumerate(img_list):
                if not isinstance(item, dict):
                    continue
                image_name, boxes_list = normalize_item_to_annots(item, fallback_image=f"img_{idx}")
                if not boxes_list:
                    continue

                minXs = [b["minX"] for b in boxes_list]
                minYs = [b["minY"] for b in boxes_list]
                maxXs = [b["maxX"] for b in boxes_list]
                maxYs = [b["maxY"] for b in boxes_list]

                agg_minX = statistics.median(minXs)
                agg_minY = statistics.median(minYs)
                agg_maxX = statistics.median(maxXs)
                agg_maxY = statistics.median(maxYs)

                centerX = (agg_minX + agg_maxX) / 2.0
                centerY = (agg_minY + agg_maxY) / 2.0
                width   = agg_maxX - agg_minX
                height  = agg_maxY - agg_minY

                rows.append({
                    "image": image_name,
                    "agg_minX": agg_minX, "agg_minY": agg_minY,
                    "agg_maxX": agg_maxX, "agg_maxY": agg_maxY,
                    "centerX": centerX, "centerY": centerY,
                    "width": width, "height": height,
                })
        else:
            # 형태 B: 루트의 각 key가 이미지 단위 객체라고 가정
            for key, item in data.items():
                if not isinstance(item, dict):
                    continue
                image_name, boxes_list = normalize_item_to_annots(item, fallback_image=str(key))
                if not boxes_list:
                    continue

                minXs = [b["minX"] for b in boxes_list]
                minYs = [b["minY"] for b in boxes_list]
                maxXs = [b["maxX"] for b in boxes_list]
                maxYs = [b["maxY"] for b in boxes_list]

                agg_minX = statistics.median(minXs)
                agg_minY = statistics.median(minYs)
                agg_maxX = statistics.median(maxXs)
                agg_maxY = statistics.median(maxYs)

                centerX = (agg_minX + agg_maxX) / 2.0
                centerY = (agg_minY + agg_maxY) / 2.0
                width   = agg_maxX - agg_minX
                height  = agg_maxY - agg_minY

                rows.append({
                    "image": image_name,
                    "agg_minX": agg_minX, "agg_minY": agg_minY,
                    "agg_maxX": agg_maxX, "agg_maxY": agg_maxY,
                    "centerX": centerX, "centerY": centerY,
                    "width": width, "height": height,
                })

    else:
        raise ValueError("지원하지 않는 최상위 JSON 구조입니다. (dict 또는 list 여야 합니다)")

    return rows

def main():
    rows = process_large_json(INPUT_JSON)
    if not rows:
        print("집계할 데이터가 없습니다. 입력 JSON 구조와 annot A/B/C 존재 여부를 확인하세요.")
        return

    os.makedirs(os.path.dirname(OUTPUT_CSV) or ".", exist_ok=True)
    fieldnames = [
        "image",
        "agg_minX", "agg_minY", "agg_maxX", "agg_maxY",
        "centerX", "centerY", "width", "height",
    ]
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(rows)

    print(f"완료! {len(rows)}개 이미지의 중앙값 박스를 {OUTPUT_CSV}에 저장했습니다.")

if __name__ == "__main__":
    main()

완료! 3710개 이미지의 중앙값 박스를 ./bbox_agg_median_per_image.csv에 저장했습니다.


json자동 중간값 필터기
